# Demo: Detecting queries with Data Movement (M05\_L02\_Demo02)

**Objective:** <span style="font-size: 11pt; font-family: Calibri, sans-serif; color: black;">The goal of this demo is to demonstrate how to detect query that has data movement with TSQL</span>

### Execute the following long-running query against the database using SSMS or another tool if it is not still running from the previous demo:

<span style="color: rgb(0, 0, 255);">SELECT</span> A.PassengerCount,  
      <span style="color: rgb(121, 94, 38);">SUM</span>(A.TripDistanceMiles<span style="color: rgb(0, 0, 0);">+</span>B.TripDistanceMiles) <span style="color: rgb(0, 0, 255);">as</span> SumTripDistance,  
      <span style="color: rgb(121, 94, 38);">AVG</span>(A.TripDistanceMiles<span style="color: rgb(0, 0, 0);">+</span>B.TripDistanceMiles) <span style="color: rgb(0, 0, 255);">as</span> AvgTripDistance  
<span style="color: rgb(0, 0, 255);">FROM</span>  dbo.Trip A, dbo.Trip B   
<span style="color: rgb(0, 0, 255);">WHERE</span> A.TripDistanceMiles <span style="color: rgb(0, 0, 0);">&gt;</span> <span style="color: rgb(9, 136, 90);">0</span> <span style="color: rgb(0, 0, 255);">AND</span> A.PassengerCount <span style="color: rgb(0, 0, 0);">&gt;</span> <span style="color: rgb(9, 136, 90);">0</span>  
<span style="color: rgb(0, 0, 255);">GROUP&nbsp;BY</span> A.PassengerCount  
<span style="color: rgb(0, 0, 255);">ORDER&nbsp;BY</span> A.PassengerCount

## Execute the following query to show the number of steps using DMS

In [18]:
--It will show the total number of steps using DMS, the total elapsed time and the DMS weight during the executions
SELECT TOP 50
		S.session_id
		,S.login_name
		,S.client_id
		,S.app_name
		,R.request_id
		,R.command request_command
		,R.[label]
		,COUNT(ST.request_id) steps_with_movement
		,SUM(ST.row_count) total_rows_moved
		,SUM(ST.total_elapsed_time) total_elapsed_dms_time
		,SUM(R.total_elapsed_time) total_elapsed_request_time
		,(SUM(ST.total_elapsed_time)*100.00)/SUM(R.total_elapsed_time) dms_time_percentage
FROM sys.dm_pdw_exec_requests R
	INNER JOIN sys.dm_pdw_request_steps ST
		ON R.request_id = ST.request_id
	INNER JOIN sys.dm_pdw_exec_sessions S
		ON R.session_id = S.session_id
WHERE (ST.location_type = 'DMS'
	OR ST.operation_type like '%Move%')
	AND R.status = 'Completed'
GROUP BY 
		S.session_id
		,S.login_name
		,S.client_id
		,S.app_name
		,R.request_id
		,R.command
		,R.[label]
HAVING SUM(ST.row_count) > 10000
ORDER BY dms_time_percentage DESC
		,total_rows_moved DESC


(0 rows affected)

Total execution time: 00:00:02.877

session_id,login_name,client_id,app_name,request_id,request_command,label,steps_with_movement,total_rows_moved,total_elapsed_dms_time,total_elapsed_request_time,dms_time_percentage


In [14]:
--Alternative version for running queries showing data moved across the system
/*
	=================================
	   Data Movement Heavy Hitters
	=================================
	The following query returns queries that are currently processing rows in DMS sorted from largest to smallest. 
    Queries that are processing very large amounts of rows or volume of data could be spilling to tempdb and impacting other queries. 
    When troubleshooting a "general slowness" issue, this query can help you find the query steps that are most likley to be impacting 
    other resources. 
*/
WITH step_data AS 
(
	SELECT 
		SUM(rows_processed) AS Step_Rows_Processed
		, SUM(Bytes_processed) AS Step_Bytes_Processed
		, request_id
		, step_index
		, status 
		, type
	FROM sys.dm_pdw_dms_workers 
	WHERE status != 'StepComplete'
	Group by 
		  request_id
		, step_index
		, status
		, type
) 
SELECT 
	 step_data.request_id
	, per.session_id
	, step_data.Step_rows_processed
	, step_data.step_bytes_processed
	, (step_data.Step_Bytes_Processed/1024/1024/1024) AS Step_GB_Processed
	, step_data.step_index
	, step_data.type
	, step_data.status as Step_status
	, per.status AS QID_Status
	, per.total_elapsed_time/1000/60 as 'QID_Elapsed_Time (min)'
	, per.command AS 'QID_Command'
	, per.resource_class
	, per.importance
FROM step_data
LEFT JOIN sys.dm_pdw_exec_requests per
ON step_data.request_id = per.request_id
WHERE per.status = 'Running'
ORDER BY step_data.Step_rows_processed DESC

(1 row affected)

Total execution time: 00:00:06.566

request_id,session_id,Step_rows_processed,step_bytes_processed,Step_GB_Processed,step_index,type,Step_status,QID_Status,QID_Elapsed_Time (min),QID_Command,resource_class,importance
QID8269,SID450,0,0,0,5,HASH_READER,Execute,Running,0,"SELECT A.PassengerCount, SUM(A.TripDistanceMiles+B.TripDistanceMiles) as SumTripDistance, AVG(A.TripDistanceMiles+B.TripDistanceMiles) as AvgTripDistance FROM dbo.Trip A, dbo.Trip B WHERE A.TripDistanceMiles > 0 AND A.PassengerCount > 0 GROUP BY A.PassengerCount ORDER BY A.PassengerCount",smallrc,normal


<span style="mso-fareast-font-family:&quot;Times New Roman&quot;;
mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin;color:#171717">Previous query should have used data movement This one shows all the query with DMS steps giving aggregate info. It returns an aggregate summarizing all the DMS steps.</span>

<span style="mso-fareast-font-family:&quot;Times New Roman&quot;;
mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin;color:#171717">Choose a query_id with DMS</span>

<span style="mso-fareast-font-family:&quot;Times New Roman&quot;;
mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin;color:#171717">In order to identify which is the most expensive DMS step run this query changing its query_id using yours:</span>

In [15]:
SELECT * FROM sys.dm_pdw_request_steps WHERE request_id = 'QID8269'

(9 rows affected)

Total execution time: 00:00:01.288

request_id,step_index,plan_node_id,operation_type,distribution_type,location_type,status,error_id,start_time,end_time,total_elapsed_time,row_count,estimated_rows,command
QID8269,0,-1,RandomIDOperation,Unspecified,Control,Complete,NULL,2022-02-24 18:31:28.520,2022-02-24 18:31:28.520,0,-1,-1,TEMP_ID_41
QID8269,1,-1,OnOperation,AllComputeNodes,Compute,Complete,NULL,2022-02-24 18:31:28.520,2022-02-24 18:31:28.550,31,-1,-1,CREATE TABLE [qtabledb].[dbo].[TEMP_ID_41] ([TripDistanceMiles] FLOAT(53) ) WITH(DISTRIBUTED_MOVE_FILE='');
QID8269,2,2,BroadcastMoveOperation,AllDistributions,Compute,Complete,NULL,2022-02-24 18:31:28.550,2022-02-24 18:31:30.910,2359,170261325,170261000,"SELECT [T1_1].[TripDistanceMiles] AS [TripDistanceMiles] FROM [wsplussynapsepool1].[dbo].[Trip] AS T1_1 OPTION (MAXDOP 4, MIN_GRANT_PERCENT = [MIN_GRANT], DISTRIBUTED_MOVE(N''), MAX_GRANT_PERCENT = [MAX_GRANT])"
QID8269,3,-1,RandomIDOperation,Unspecified,Control,Complete,NULL,2022-02-24 18:31:30.910,2022-02-24 18:31:30.910,0,-1,-1,TEMP_ID_42
QID8269,4,-1,OnOperation,AllDistributions,Compute,Complete,NULL,2022-02-24 18:31:30.910,2022-02-24 18:31:31.067,156,-1,-1,"CREATE TABLE [qtabledb].[dbo].[TEMP_ID_42] ([PassengerCount] INT NOT NULL, [col] FLOAT(53), [col1] BIGINT, [col2] FLOAT(53) NOT NULL ) WITH(DISTRIBUTED_MOVE_FILE='');"
QID8269,5,35,ShuffleMoveOperation,AllDistributions,Compute,Running,NULL,2022-02-24 18:31:31.067,NULL,143937,0,8,"SELECT [T1_1].[PassengerCount] AS [PassengerCount], [T1_1].[col1] AS [col], [T1_1].[col2] AS [col1], [T1_1].[col] AS [col2] FROM (SELECT ISNULL([T2_1].[col2], CONVERT (FLOAT, 0, 0)) AS [col], [T2_1].[PassengerCount] AS [PassengerCount], [T2_1].[col] AS [col1], [T2_1].[col1] AS [col2] FROM (SELECT SUM(([T3_2].[TripDistanceMiles] + [T3_1].[TripDistanceMiles])) AS [col], COUNT_BIG(([T3_2].[TripDistanceMiles] + [T3_1].[TripDistanceMiles])) AS [col1], SUM(([T3_2].[TripDistanceMiles] + [T3_1].[TripDistanceMiles])) AS [col2], [T3_2].[PassengerCount] AS [PassengerCount] FROM [qtabledb].[dbo].[TEMP_ID_41] AS T3_1 INNER JOIN (SELECT [T4_1].[TripDistanceMiles] AS [TripDistanceMiles], [T4_1].[PassengerCount] AS [PassengerCount] FROM [wsplussynapsepool1].[dbo].[Trip] AS T4_1 WHERE (([T4_1].[TripDistanceMiles] > CAST (0.00000000000000000E+000 AS FLOAT)) AND ([T4_1].[PassengerCount] > CAST ((0) AS INT)))) AS T3_2 ON (0 = 0) GROUP BY [T3_2].[PassengerCount]) AS T2_1) AS T1_1 OPTION (MAXDOP 4, MIN_GRANT_PERCENT = [MIN_GRANT], DISTRIBUTED_MOVE(N''), MAX_GRANT_PERCENT = [MAX_GRANT])"
QID8269,6,44,ReturnOperation,AllDistributions,Compute,Pending,NULL,NULL,NULL,NULL,0,0,"SELECT [T1_1].[PassengerCount] AS [PassengerCount], [T1_1].[col1] AS [col], [T1_1].[col] AS [col1] FROM (SELECT CASE WHEN ([T2_1].[col] = CAST ((0) AS BIGINT)) THEN CAST (NULL AS FLOAT) ELSE ([T2_1].[col1] / CONVERT (FLOAT, [T2_1].[col], 0)) END AS [col], [T2_1].[PassengerCount] AS [PassengerCount], [T2_1].[col2] AS [col1] FROM (SELECT ISNULL([T3_1].[col1], CONVERT (BIGINT, 0, 0)) AS [col], ISNULL([T3_1].[col2], CONVERT (FLOAT, 0, 0)) AS [col1], [T3_1].[PassengerCount] AS [PassengerCount], [T3_1].[col] AS [col2] FROM (SELECT SUM([T4_1].[col]) AS [col], SUM([T4_1].[col1]) AS [col1], SUM([T4_1].[col2]) AS [col2], [T4_1].[PassengerCount] AS [PassengerCount] FROM [qtabledb].[dbo].[TEMP_ID_42] AS T4_1 GROUP BY [T4_1].[PassengerCount]) AS T3_1) AS T2_1) AS T1_1 ORDER BY [T1_1].[PassengerCount] ASC OPTION (MAXDOP 4, MIN_GRANT_PERCENT = [MIN_GRANT], MAX_GRANT_PERCENT = [MAX_GRANT])"
QID8269,7,-1,OnOperation,AllDistributions,Compute,Pending,NULL,NULL,NULL,NULL,-1,-1,DROP TABLE [qtabledb].[dbo].[TEMP_ID_42]
QID8269,8,-1,OnOperation,AllComputeNodes,Compute,Pending,NULL,NULL,NULL,NULL,-1,-1,DROP TABLE [qtabledb].[dbo].[TEMP_ID_41]


If you want to go one level deeper to see the distribution-level work that is happening you can query DMS workers. Copy the Request ID from the query above and the step index from the Data Movement step you are interested in and paste them into the following query:

In [16]:
SELECT * FROM sys.dm_pdw_dms_workers
WHERE request_id = 'QID8269'
AND step_index = 5

Analysis: This query will generally retrun 120 rows because it is showing 60 distributions, each with a reader and a writer. The readers are generally the interesting step as they are running a query and gerneating the rows, while the writers are just writing what comes to them into each distribution. It's interesting at this level to look for any skew in the readers or writers as well as if the currently running step is actually running on all distributions or if just 1 or a few are still executing. Running this multiple times you will see rows processed increase.